# Lab Exercise 04 (Optional): Fourier Transform and Filtering
## Objective:

In this lab, you will:

1. Generate a synthetic Gaussian ripple pattern.
2. Apply the Fast Fourier Transform (FFT) to the synthetic data and observe its frequency domain representation.
3. Add regular horizontal scanlines to the data to simulate periodic noise.
4. Apply FFT to the noisy data and compare the results
5. Apply a filter in the frequency domain to remove the scanlines and restore the cleaned image.
   
**Answer** this notebook by updating the **Answers for Part. x** cell by **double clicking and typing your answers in a cell**. After finishing this notebook, upload in your github repository (**meteo203-2425-lastname/exercises/exercise_04_fouriertransform_noisefiltering.ipynb**)

### Part 1: From 2D sine wave to 3D ripple

In this exercise, we simulate a 2D sinusoidal ripple (like ripples in a pond).

- The parameter $f$ controls the frequency of the ripples (how many rings you see).
- Larger $f$ → tighter, more frequent ripples.
- Smaller $f$ → wider, smoother ripples.

Try this:
- Change the value of f and re-run the cell.
- Observe how the ripple spacing changes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create a grid of points (like an image matrix)
rows, cols = 256, 256  # Size of the synthetic image
x = np.linspace(-5, 5, cols)
y = np.linspace(-5, 5, rows)
X, Y = np.meshgrid(x, y)
# Generate ripple pattern using a 2D sinusoidal function
f = 10
ripple = np.sin(np.sqrt(X**2 + Y**2) * f)  # Adjust frequency of ripples with f

plt.imshow(ripple, cmap='gray')
plt.title('Ripple Pattern')
plt.axis('off')
plt.show()


#### Questions:

1. What kinds of real-world meteorological or oceanographic phenomena could this ripple pattern represent? (Hint: Think about waves that spread out in space — surface waves, atmospheric wave trains, etc.)


---

### Part 2: Ripples in Frequency Space (FFT)

The ripple pattern you generated looks simple, but it’s actually made up of many tiny waves.

To “see” those hidden waves, we use the Fourier Transform (FFT). This turns the ripple from space domain (the picture you plotted) into frequency domain (what frequencies make up the picture). 

- Bright spots in the FFT image correspond to strong frequency components.
- The farther from the center, the higher the frequency (more ripples per unit distance).

Try this:
- Run the code below once. What do you observe?
- Change f amdd re-run this FFT.
- How does the FFT pattern change?


In [ ]:
from numpy.fft import fft2, fftshift

# Define the ripple
# Generate ripple pattern using a 2D sinusoidal function
f = 5
ripple = np.sin(np.sqrt(X**2 + Y**2) * f)  # Adjust frequency of ripples with f

# 2D Fourier Transform of the ripple
fft_ripple = np.fft.fft2(ripple)
fft_shift = np.fft.fftshift(fft_ripple)  # Shift zero frequency to center
magnitude_spectrum = np.abs(fft_shift)

plt.figure(figsize=(12,5))

# Original ripple
plt.subplot(1,2,1)
plt.imshow(ripple, cmap='gray')
plt.title('Ripple Pattern')
plt.axis('off')

# Fourier spectrum
plt.subplot(1,2,2)
plt.imshow(np.log1p(magnitude_spectrum), cmap='inferno')
plt.title('FFT (Frequency Space)')
plt.axis('off')

plt.show()


#### Questions:

2. How many bright “rings” or spots do you see in the frequency plot?
3. Why do they move outward when you increase f?
4. Can you think of a real-world example where scientists might want to look at the “hidden frequencies” of waves (in the ocean, atmosphere, or climate data)?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create a grid of points (like an image matrix)
rows, cols = 256, 256  # Size of the synthetic image
x = np.linspace(-5, 5, cols)
y = np.linspace(-5, 5, rows)
X, Y = np.meshgrid(x, y)

# Generate ripple pattern using a 2D sinusoidal function
f = 5
ripple = np.sin(np.sqrt(X**2 + Y**2) * f)  # Adjust frequency of ripples with multiplier

# Generate a 2D Gaussian envelope
sigma = 1.5  # Standard deviation of the Gaussian
gaussian_envelope = np.exp(-(X**2 + Y**2) / (2 * sigma**2))

# Apply the Gaussian envelope to the ripple pattern
gaussian_ripple = ripple * gaussian_envelope

# Plot the synthetic Gaussian-weighted ripple data
plt.imshow(gaussian_ripple, cmap='gray')
plt.title('Gaussian-Weighted Ripple Pattern')
plt.axis('off')
plt.show()


#### Questions:

1. How does the Gaussian weighting ($sigma$) affect the appearance of the ripple pattern?
2. In what types of meteorological or geophysical data might you encounter a similar Gaussian distribution?

In [ ]:
from numpy.fft import fft2, fftshift

# Apply 2D FFT to the Gaussian-weighted ripple data
fft_gaussian_ripple = fft2(gaussian_ripple)

# Shift the zero-frequency component to the center of the spectrum
fft_gaussian_ripple_shifted = fftshift(fft_gaussian_ripple)

# Compute the magnitude spectrum
magnitude_spectrum_gaussian_ripple = np.log(np.abs(fft_gaussian_ripple_shifted) + 1)

# Plot the magnitude spectrum of the FFT of the Gaussian ripple
plt.imshow(magnitude_spectrum_gaussian_ripple, cmap='gray')
plt.title('FFT Magnitude Spectrum of Gaussian Ripple Pattern')
plt.axis('off')
plt.show()


In [ ]:
from numpy.fft import ifft2, ifftshift

# Perform inverse FFT shift to move the zero-frequency component back to its original position
ifft_shifted = ifftshift(fft_gaussian_ripple_shifted)

# Apply the inverse FFT to restore the image from the frequency domain
restored_gaussian_image = np.abs(ifft2(ifft_shifted))

# Display the restored image
plt.imshow(restored_gaussian_image, cmap='gray')
plt.title('Restored Gaussian Ripple Image Using IFFT')
plt.axis('off')
plt.show()


In [ ]:
from numpy.fft import ifft2, ifftshift

# Perform inverse FFT shift to move the zero-frequency component back to its original position
ifft_shifted = ifftshift(fft_gaussian_ripple_shifted)

# Apply the inverse FFT to restore the image from the frequency domain
restored_gaussian_image = np.abs(ifft2(ifft_shifted))

# Display the restored image
plt.imshow(restored_gaussian_image, cmap='gray')
plt.title('Restored Gaussian Ripple Image Using IFFT')
plt.axis('off')
plt.show()


---

### Part 3 — Add “scanlines” (striping) like a satellite artifact
#### Task: Add Regular Horizontal Scanlines

Real sensors often produce striping (scanlines): periodic lines across the image due to detector gain differences or readout noise.
We’ll simulate this as a periodic vertical stripe (varying along x, repeated along y).

- $stripe\_freq$ controls how many stripes across the image.

- $stripe\_amp$ controls their strength.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- base ripple from Part 1 ---
rows, cols = 256, 256
x = np.linspace(-5, 5, cols)
y = np.linspace(-5, 5, rows)
X, Y = np.meshgrid(x, y)

f = 5  # ripple frequency
ripple = np.sin(np.sqrt(X**2 + Y**2) * f)

# --- add scanlines (vertical stripes) ---
stripe_freq = 12          # number of stripes across the width
stripe_amp  = 0.35        # strength of the stripes

col_idx = np.arange(cols)
vertical_stripe = np.sin(2*np.pi * stripe_freq * col_idx / cols)  # 1D pattern across x
vertical_stripe_2d = np.tile(vertical_stripe, (rows, 1))          # repeat down the rows

ripple_striped = ripple + stripe_amp * vertical_stripe_2d

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(ripple, cmap='gray')
plt.title('Clean Ripple')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(ripple_striped, cmap='gray')
plt.title('Ripple + Scanlines')
plt.axis('off')
plt.show()


---

### Part 4 — Look at the stripes in frequency space (FFT)

In frequency space, periodic stripes appear as narrow spikes along the axes.

- Vertical stripes in the image → spikes along the horizontal (kx) axis in the FFT.
- The distance of spikes from the center ≈ the stripe frequency.

In [ ]:
def show_fft(img, title='FFT (log magnitude)'):
    F = np.fft.fft2(img)
    F_shift = np.fft.fftshift(F)
    mag = np.log1p(np.abs(F_shift))
    plt.imshow(mag, cmap='inferno')
    plt.title(title)
    plt.axis('off')
    return F, F_shift

plt.figure(figsize=(12,4))
plt.subplot(1,2,1); _=show_fft(ripple, 'Clean Ripple FFT')
plt.subplot(1,2,2); _=show_fft(ripple_striped, 'Striped Ripple FFT')
plt.show()


### Questions:

1. Compare the two figures in frequency space above. Can you tell the difference? What do the small bright dots represent?

---
### Part 5 — Build a notch filter to remove the stripe spikes
We’ll build a notch filter that zeros out the stripe spikes in FFT space.
- Use $stripe\_freq$ to locate the spikes at ±kx.
- $notch\_halfwidth$ sets how wide the notch is (a few pixels is usually enough).
Keep the center (DC) untouched.

In [ ]:
# Frequency coordinates (in pixels, centered)
ky = np.fft.fftshift(np.fft.fftfreq(rows)) * rows
kx = np.fft.fftshift(np.fft.fftfreq(cols)) * cols
KX, KY = np.meshgrid(kx, ky)

# Expected spike positions for vertical stripes (appear along kx axis near ±stripe_freq_alias)
# Because we constructed stripes as sin(2π * stripe_freq * x / cols), the spike index is near ±stripe_freq
target = stripe_freq          # approximate location in kx (pixels)
notch_halfwidth = 2           # +/- pixels around the spike to zero out
dc_keep_radius = 3            # keep a small circle around DC (0,0)

# Start with all-pass mask
mask = np.ones((rows, cols), dtype=float)

# Helper: zero a vertical band around kx = +target and -target (|kx - target| <= hw)
def notch_out(mask, KX, target, hw):
    band_pos = np.abs(KX - target) <= hw
    band_neg = np.abs(KX + target) <= hw
    mask[band_pos] = 0.0
    mask[band_neg] = 0.0

# Apply notches (avoid wiping the DC neighborhood)
notch_out(mask, KX, target, notch_halfwidth)

# Optional: protect a tiny DC disk
dc = (KX**2 + KY**2) <= dc_keep_radius**2
mask[dc] = 1.0

plt.figure(figsize=(5,4))
plt.imshow(mask, cmap='gray')
plt.title('Notch Mask')
plt.axis('off')
plt.show()


### Part 6 — Filter in frequency space, then IFFT to recover

In [ ]:
# FFT of striped image
F = np.fft.fft2(ripple_striped)
F_shift = np.fft.fftshift(F)

# Apply mask
F_shift_filtered = F_shift * mask

# Back to spatial domain
F_filtered = np.fft.ifftshift(F_shift_filtered)
recovered = np.fft.ifft2(F_filtered).real

plt.figure(figsize=(12,6))
plt.subplot(2,3,1); plt.imshow(ripple_striped, cmap='gray'); plt.title('Input (Striped)'); plt.axis('off')
plt.subplot(2,3,2); plt.imshow(np.log1p(np.abs(F_shift)), cmap='inferno'); plt.title('FFT: Striped'); plt.axis('off')
plt.subplot(2,3,3); plt.imshow(mask, cmap='gray'); plt.title('Notch Mask'); plt.axis('off')
plt.subplot(2,3,4); plt.imshow(recovered, cmap='gray'); plt.title('Recovered (IFFT)'); plt.axis('off')
plt.subplot(2,3,5); plt.imshow(np.log1p(np.abs(F_shift_filtered)), cmap='inferno'); plt.title('FFT: Filtered'); plt.axis('off')
plt.subplot(2,3,6); plt.imshow(ripple, cmap='gray'); plt.title('Reference (Clean)'); plt.axis('off')
plt.tight_layout()
plt.show()


### Questions:

2. Why is a vertical band-stop filter effective in removing horizontal scanlines in the frequency domain?
3. How might this approach of Fourier filtering be applied in real-world scenarios, such as removing periodic noise from meteorological or satellite data?

In [ ]:
from numpy.fft import ifft2, ifftshift

# Apply inverse shift to move the frequency components back
ifft_shifted = ifftshift(filtered_fft_gaussian_scanlines)

# Apply inverse FFT to get the filtered image
filtered_gaussian_image = np.abs(ifft2(ifft_shifted))

# Normalize the restored image (0 to 1 range)
filtered_gaussian_image_normalized = (filtered_gaussian_image - np.min(filtered_gaussian_image)) / (np.max(filtered_gaussian_image) - np.min(filtered_gaussian_image))

# Optionally, brighten the image by scaling pixel values (e.g., multiply by a constant)
brightness_factor = 1.5
filtered_gaussian_image_brightened = np.clip(filtered_gaussian_image_normalized * brightness_factor, 0, 1)

# Display the brightened restored image
plt.imshow(filtered_gaussian_image_brightened, cmap='gray')
plt.title('Restored and Brightened Gaussian Ripple Image After Removing Scanlines')
plt.axis('off')
plt.show()
